In [74]:
with open("input/day7.txt", "r") as f:
    data = f.readlines()
    
data = [x.strip() for x in data]
data[:10]

['$ cd /',
 '$ ls',
 'dir fcqv',
 'dir fcv',
 '72939 hdpgfcwd',
 '236918 jlncjqh.csz',
 'dir jvwfwrg',
 'dir tzwpllhq',
 'dir vglf',
 '28586 wzljr.zvp']

In [106]:
class File:
    def __init__(self, name: str, size: int, parent: Directory):
        self.name = name
        self.size = size
        self.parent = parent

class Directory:
    def __init__(self, name: str, parent: Directory, size = 0, depth = 0):
        self.name = name
        self.parent = parent
        self.children = []
        self.files = []
        self.size = size
        self.depth = depth
        
    def updateSize(self, size):
        self.size += size
        if self.parent is None:
            return
        self.parent.updateSize(size)
    
    def addChildDir(self, name):
        child_names = [x.name for x in self.children]
        if name not in child_names:
            child = Directory(name, self, depth=self.depth + 1)
            self.children.append(child)
    
    def addFile(self, file: File):
        self.files.append(file)
        self.updateSize(file.size)
    
    def getChildDir(self, name):
        for child in self.children:
            if child.name == name:
                return child
        return False
    
    def __str__(self):
        indent = "  " * self.depth
        return f"{indent}{self.name} - {self.size}"
        
data_p1 = data

root_dir = Directory("/", None)
current_dir = root_dir

for line in data_p1[1:]:
    command = line.split()
    
    if command[0] == "dir":
        current_dir.addChildDir(command[1])
    
    elif command[0] == "$":
        if command[1] == "ls":
            continue
        elif command[1] == "cd":
            if command[2] == "..":
                current_dir = current_dir.parent
            else:
                name = command[2]
                current_dir.addChildDir(name)
                current_dir = current_dir.getChildDir(name)
    
    # it's a file
    else:
        file = File(command[1], int(command[0]), current_dir)
        current_dir.addFile(file)

def printDirectory(d, indent):
    print(d)
    for c in d.children:
        printDirectory(c, indent + 1)
 
#printDirectory(root_dir, 0)

answer = 0
def calculateAnswer(d):
    global answer
    if d.size <= 100000:
        answer += d.size
    for c in d.children:
        calculateAnswer(c)

calculateAnswer(root_dir)
print(f"Sum of folders that are larger than {100000}: {answer}")

Sum of folders that are larger than 100000: 1182909


In [104]:
disk_size = 70000000
used_space = root_dir.size
free_space = disk_size - used_space
needed_space = 30000000

amount_to_delete = needed_space - free_space
amount_to_delete

dir_sizes_that_can_be_deleted = []

def getSizes(d):
    global dir_sizes_that_can_be_deleted
    if d.size >= amount_to_delete:
        dir_sizes_that_can_be_deleted.append(d.size)
    for c in d.children:
        getSizes(c)

print(f"Amount needed to be freeed up: {amount_to_delete}")
getSizes(root_dir)
print(f"Smallest folder size that can be deleted: {sorted(dir_sizes_that_can_be_deleted)[0]}")

Amount needed to be freeed up: 2677139
Smallest folder size that can be deleted: 2832508
